# SPIRE-NEP master catalogue

This notebook presents the merge of the various pristine catalogues to produce HELP mater catalogue on SPIRE-NEP.

In [1]:
from herschelhelp_internal import git_version
print("This notebook was run with herschelhelp_internal version: \n{}".format(git_version()))

This notebook was run with herschelhelp_internal version: 
04829ed (Thu Nov 2 16:57:19 2017 +0000) [with local modifications]


In [2]:
%matplotlib inline
#%config InlineBackend.figure_format = 'svg'

import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))

import os
import time

from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.table import Column, Table
import numpy as np
from pymoc import MOC

from herschelhelp_internal.masterlist import merge_catalogues, nb_merge_dist_plot, specz_merge
from herschelhelp_internal.utils import coords_to_hpidx, ebv, gen_help_id, inMoc

/Users/rs548/anaconda/envs/herschelhelp_internal/lib/python3.6/site-packages/seaborn/apionly.py:6: UserWarning: As seaborn no longer sets a default style on import, the seaborn.apionly module is deprecated. It will be removed in a future version.
  warnings.warn(msg, UserWarning)


In [3]:
TMP_DIR = os.environ.get('TMP_DIR', "./data_tmp")
OUT_DIR = os.environ.get('OUT_DIR', "./data")
SUFFIX = os.environ.get('SUFFIX', time.strftime("_%Y%m%d"))

try:
    os.makedirs(OUT_DIR)
except FileExistsError:
    pass

## I - Reading the prepared pristine catalogues

In [4]:
ps1 = Table.read("{}/PS1.fits".format(TMP_DIR))

## II - Merging tables

We first merge the optical catalogues and then add the infrared ones: DECaLS, HSC, KIDS, PanSTARRS, UKIDSS-LAS, and VISTA-VIKING.

At every step, we look at the distribution of the distances to the nearest source in the merged catalogue to determine the best crossmatching radius.

### DECaLS

In [5]:
master_catalogue = ps1
master_catalogue['ps1_ra'].name = 'ra'
master_catalogue['ps1_dec'].name = 'dec'

### Cleaning

When we merge the catalogues, astropy masks the non-existent values (e.g. when a row comes only from a catalogue and has no counterparts in the other, the columns from the latest are masked for that row). We indicate to use NaN for masked values for floats columns, False for flag columns and -1 for ID columns.

In [6]:
for col in master_catalogue.colnames:
    if "m_" in col or "merr_" in col or "f_" in col or "ferr_" in col or "stellarity" in col:
        master_catalogue[col].fill_value = np.nan
    elif "flag" in col:
        master_catalogue[col].fill_value = 0
    elif "id" in col:
        master_catalogue[col].fill_value = -1
        
master_catalogue = master_catalogue.filled()

In [7]:
master_catalogue[:10].show_in_notebook()

idx,ps1_id,ra,dec,m_ap_gpc1_g,merr_ap_gpc1_g,m_gpc1_g,merr_gpc1_g,m_ap_gpc1_r,merr_ap_gpc1_r,m_gpc1_r,merr_gpc1_r,m_ap_gpc1_i,merr_ap_gpc1_i,m_gpc1_i,merr_gpc1_i,m_ap_gpc1_z,merr_ap_gpc1_z,m_gpc1_z,merr_gpc1_z,m_ap_gpc1_y,merr_ap_gpc1_y,m_gpc1_y,merr_gpc1_y,f_ap_gpc1_g,ferr_ap_gpc1_g,f_gpc1_g,ferr_gpc1_g,flag_gpc1_g,f_ap_gpc1_r,ferr_ap_gpc1_r,f_gpc1_r,ferr_gpc1_r,flag_gpc1_r,f_ap_gpc1_i,ferr_ap_gpc1_i,f_gpc1_i,ferr_gpc1_i,flag_gpc1_i,f_ap_gpc1_z,ferr_ap_gpc1_z,f_gpc1_z,ferr_gpc1_z,flag_gpc1_z,f_ap_gpc1_y,ferr_ap_gpc1_y,f_gpc1_y,ferr_gpc1_y,flag_gpc1_y,ps1_flag_cleaned,ps1_flag_gaia
,,deg,deg,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,190722653882417246,265.388181797,68.9387146457,19.2134990692,0.00342800002545,nan,nan,nan,4.99999987369e-05,nan,nan,18.8502006531,0.00138799997512,nan,nan,18.7789993286,0.00228199991398,nan,nan,19.0373001099,0.00218400009908,nan,nan,74.9204491495,0.236546686447,nan,nan,False,nan,nan,nan,nan,False,104.693504787,0.13383967616,nan,nan,False,111.789308198,0.234958722546,nan,nan,False,88.1211093989,0.177258997917,nan,nan,False,False,3
1,190482652731951974,265.273177477,68.7343304857,13.1686000824,0.000174000000698,13.2056999207,0.000456999987364,12.7111997604,0.000100999997812,12.6421003342,0.000666000007186,12.8554000854,8.80000006873e-05,12.869799614,0.000577000027988,12.5587997437,0.000125000005937,12.6094999313,0.000254999991739,12.5476999283,0.000113000001875,12.5921001434,0.000345999986166,19613.7198148,3.14329322952,18954.8360235,7.97832823965,False,29889.5994995,2.780463128,31853.6985917,19.5393350595,False,26172.1840563,2.12128158281,25827.3682268,13.7256104424,False,34393.7953297,3.95973220896,32824.6442134,7.70931646802,False,34747.2171764,3.61638083806,33354.923016,10.6294722894,False,True,3
2,190722653912264533,265.391174867,68.9364847957,12.7490997314,0.00017499999376,12.785200119,0.000911000010092,12.4386997223,0.000100999997812,12.4645004272,0.000361000013072,12.2482995987,6.60000005155e-05,12.288599968,0.000490000005811,12.2592000961,0.000113000001875,12.29529953,0.000292999990052,12.1958999634,9.40000027185e-05,12.1674003601,0.000339999998687,28864.2387026,4.65236543712,27920.2917696,23.4268563325,False,38416.7048576,3.57369229244,37514.5575839,12.4733390292,False,45780.4608845,2.78291396081,44112.3315367,19.9081900508,False,45323.1370016,4.71708923743,43840.9733774,11.8310550087,False,48044.0947342,4.15952329473,49321.9065446,15.4452325408,False,False,3
3,190672651606062514,265.160522787,68.8931333257,12.815199852,0.000167000005604,12.9259004593,0.000843000016175,12.7364997864,0.000109000000521,13.3079996109,0.00110700004734,15.7264995575,0.000335999997333,nan,nan,12.2652997971,0.000119999996969,nan,nan,16.6851997375,0.000732999993488,nan,nan,27159.3929856,4.17745927686,24526.7393785,19.0433381536,False,29201.1587594,2.93158347859,17250.4391546,17.5882842232,False,1859.51710233,0.575459986215,nan,nan,False,45069.2240732,4.98123460235,nan,nan,False,768.988960157,0.519158245177,nan,nan,False,True,3
4,190622643128955464,264.312914227,68.8539061457,nan,nan,13.0476999283,0.00121300003957,12.7306995392,0.000109000000521,12.7356996536,0.000487999990582,nan,nan,13.108499527,0.00134499999695,12.265999794,0.000119999996969,12.3839998245,0.000560000014957,12.206199646,9.300000238e-05,12.241399765,0.000284999987343,nan,nan,21924.0118777,24.4938200998,False,29357.5753567,2.94728656476,29222.6864701,13.1345631284,False,nan,nan,20730.0423443,25.6801852675,False,45040.1763622,4.97802413076,40401.740067,20.8383721043,False,47590.4861874,4.07641866348,46072.3212757,12.0937396422,False,False,3
5,190412643390086959,264.339006877,68.6801702857,12.9955997467,0.000155999994604,13.0321998596,0.000372999988031,12.8657999039,0.000110000000859,12.8879995346,0.000265000009676,12.7673997879,8.09999983176e-05,12.7856998444,0.00034900000901,12.6052999496,0.000125999999,12.6520004272,0.000241999994614,12.5761003494,0.000113000001875,12.6241998672,0.000371000001905,23001.7088163,3.30491553611,22239.245863,7.64

## III - Merging flags and stellarity

Each pristine catalogue contains a flag indicating if the source was associated to a another nearby source that was removed during the cleaning process.  We merge these flags in a single one.

In [8]:
flag_cleaned_columns = [column for column in master_catalogue.colnames
                        if 'flag_cleaned' in column]

flag_column = np.zeros(len(master_catalogue), dtype=bool)
for column in flag_cleaned_columns:
    flag_column |= master_catalogue[column]
    
master_catalogue.add_column(Column(data=flag_column, name="flag_cleaned"))
master_catalogue.remove_columns(flag_cleaned_columns)

Each pristine catalogue contains a flag indicating the probability of a source being a Gaia object (0: not a Gaia object, 1: possibly, 2: probably, 3: definitely).  We merge these flags taking the highest value.

In [9]:
flag_gaia_columns = [column for column in master_catalogue.colnames
                     if 'flag_gaia' in column]

master_catalogue.add_column(Column(
    data=np.max([master_catalogue[column] for column in flag_gaia_columns], axis=0),
    name="flag_gaia"
))
master_catalogue.remove_columns(flag_gaia_columns)

Each prisitine catalogue may contain one or several stellarity columns indicating the probability (0 to 1) of each source being a star.  We merge these columns taking the highest value.

In [11]:
# No ps1 stellarity
#stellarity_columns = [column for column in master_catalogue.colnames
#                      if 'stellarity' in column]
#
#master_catalogue.add_column(Column(
#    data=np.nanmax([master_catalogue[column] for column in stellarity_columns], axis=0),
#    name="stellarity"
#))
#master_catalogue.remove_columns(stellarity_columns)

## IV - Adding E(B-V) column

In [12]:
master_catalogue.add_column(
    ebv(master_catalogue['ra'], master_catalogue['dec'])
)

## V - Adding HELP unique identifiers and field columns

In [13]:
master_catalogue.add_column(Column(gen_help_id(master_catalogue['ra'], master_catalogue['dec']),
                                   name="help_id"))
master_catalogue.add_column(Column(np.full(len(master_catalogue), "SPIRE-NEP", dtype='<U18'),
                                   name="field"))

In [14]:
# Check that the HELP Ids are unique
if len(master_catalogue) != len(np.unique(master_catalogue['help_id'])):
    print("The HELP IDs are not unique!!!")
else:
    print("OK!")

OK!


## VI - Cross-matching with spec-z catalogue

In [15]:
#specz =  Table.read("../../dmu23/dmu23_GAMA-12/data/GAMA12-specz-v2-PRIVATE.csv")

In [16]:
#specz.rename_column('RA', 'ra')
#specz.rename_column('DEC', 'dec')
#specz.rename_column('OBJID', 'specz_id')
#specz.rename_column('Z_SPEC', 'z_spec')
#specz.rename_column('Z_SOURCE', 'z_source')
#specz.rename_column('Z_QUAL', 'z_qual')
#specz.rename_column('REL', 'rel')
#specz.rename_column('AGN', 'agn')

In [17]:
#specz[:10].show_in_notebook()

In [18]:
#nb_merge_dist_plot(
#    SkyCoord(master_catalogue['ra'], master_catalogue['dec']),
#    SkyCoord(specz['ra'] * u.deg, specz['dec'] * u.deg)
#)

In [19]:
#master_catalogue = specz_merge(master_catalogue, specz, radius=1. * u.arcsec)

## VII - Choosing between multiple values for the same filter

We only have PanSTARRS on SPIRE-NEP

## VIII.a Wavelength domain coverage

We add a binary `flag_optnir_obs` indicating that a source was observed in a given wavelenght domain:

- 1 for observation in optical;
- 2 for observation in near-infrared;
- 4 for observation in mid-infrared (IRAC).

It's an integer binary flag, so a source observed both in optical and near-infrared by not in mid-infrared would have this flag at 1 + 2 = 3.

*Note 1: The observation flag is based on the creation of multi-order coverage maps from the catalogues, this may not be accurate, especially on the edges of the coverage.*

*Note 2: Being on the observation coverage does not mean having fluxes in that wavelength domain. For sources observed in one domain but having no flux in it, one must take into consideration de different depths in the catalogue we are using.*

In [20]:
ps1_moc = MOC(filename="../../dmu0/dmu0_PanSTARRS1-3SS/data/PanSTARRS1-3SS_SPIRE-NEP_MOC.fits")

In [21]:
was_observed_optical = inMoc(
    master_catalogue['ra'], master_catalogue['dec'],
    ps1_moc) 

was_observed_nir = np.zeros(len(master_catalogue), dtype=bool)

was_observed_mir = np.zeros(len(master_catalogue), dtype=bool)


In [22]:
master_catalogue.add_column(
    Column(
        1 * was_observed_optical + 2 * was_observed_nir + 4 * was_observed_mir,
        name="flag_optnir_obs")
)

## VIII.b Wavelength domain detection

We add a binary `flag_optnir_det` indicating that a source was detected in a given wavelenght domain:

- 1 for detection in optical;
- 2 for detection in near-infrared;
- 4 for detection in mid-infrared (IRAC).

It's an integer binary flag, so a source detected both in optical and near-infrared by not in mid-infrared would have this flag at 1 + 2 = 3.

*Note 1: We use the total flux columns to know if the source has flux, in some catalogues, we may have aperture flux and no total flux.*

To get rid of artefacts (chip edges, star flares, etc.) we consider that a source is detected in one wavelength domain when it has a flux value in **at least two bands**. That means that good sources will be excluded from this flag when they are on the coverage of only one band.

In [23]:
# SpARCS is a catalogue of sources detected in r (with fluxes measured at 
# this prior position in the other bands).  Thus, we are only using the r
# CFHT band.
# Check to use catalogue flags from HSC and PanSTARRS.
nb_optical_flux = (
    1 * ~np.isnan(master_catalogue['f_gpc1_g']) +
    1 * ~np.isnan(master_catalogue['f_gpc1_r']) +
    1 * ~np.isnan(master_catalogue['f_gpc1_i']) +
    1 * ~np.isnan(master_catalogue['f_gpc1_z']) +
    1 * ~np.isnan(master_catalogue['f_gpc1_y']) 
)

nb_nir_flux = np.zeros(len(master_catalogue), dtype=float)

nb_mir_flux = np.zeros(len(master_catalogue), dtype=float)

In [24]:
has_optical_flux = nb_optical_flux >= 2
has_nir_flux = nb_nir_flux >= 2
has_mir_flux = nb_mir_flux >= 2

master_catalogue.add_column(
    Column(
        1 * has_optical_flux + 2 * has_nir_flux + 4 * has_mir_flux,
        name="flag_optnir_det")
)

## IX - Cross-identification table

We are producing a table associating to each HELP identifier, the identifiers of the sources in the pristine catalogue. This can be used to easily get additional information from them.

For convenience, we also cross-match the master list with the SDSS catalogue and add the objID associated with each source, if any. **TODO: should we correct the astrometry with respect to Gaia positions?**

In [25]:
#
# Addind SDSS ids
#
sdss = Table.read("../../dmu0/dmu0_SDSS-DR13/data/SDSS-DR13_SPIRE-NEP.fits")['objID', 'ra', 'dec']
sdss_coords = SkyCoord(sdss['ra'] * u.deg, sdss['dec'] * u.deg)
idx_ml, d2d, _ = sdss_coords.match_to_catalog_sky(SkyCoord(master_catalogue['ra'], master_catalogue['dec']))
idx_sdss = np.arange(len(sdss))

# Limit the cross-match to 1 arcsec
mask = d2d <= 1. * u.arcsec
idx_ml = idx_ml[mask]
idx_sdss = idx_sdss[mask]
d2d = d2d[mask]
nb_orig_matches = len(idx_ml)

# In case of multiple associations of one master list object to an SDSS object, we keep only the
# association to the nearest one.
sort_idx = np.argsort(d2d)
idx_ml = idx_ml[sort_idx]
idx_sdss = idx_sdss[sort_idx]
_, unique_idx = np.unique(idx_ml, return_index=True)
idx_ml = idx_ml[unique_idx]
idx_sdss = idx_sdss[unique_idx]
print("{} master list rows had multiple associations.".format(nb_orig_matches - len(idx_ml)))

# Adding the ObjID to the master list
master_catalogue.add_column(Column(data=np.full(len(master_catalogue), -1, dtype='>i8'), name="sdss_id"))
master_catalogue['sdss_id'][idx_ml] = sdss['objID'][idx_sdss]

14 master list rows had multiple associations.


In [26]:
id_names = []
for col in master_catalogue.colnames:
    if '_id' in col:
        id_names += [col]
    if '_intid' in col:
        id_names += [col]
        
print(id_names)

['ps1_id', 'help_id', 'sdss_id']


In [27]:
master_catalogue[id_names].write(
    "{}/master_list_cross_ident_spire-nep{}.fits".format(OUT_DIR, SUFFIX), overwrite=True)
id_names.remove('help_id')
master_catalogue.remove_columns(id_names)

## X - Adding HEALPix index

We are adding a column with a HEALPix index at order 13 associated with each source.

In [28]:
master_catalogue.add_column(Column(
    data=coords_to_hpidx(master_catalogue['ra'], master_catalogue['dec'], order=13),
    name="hp_idx"
))

## XI - Saving the catalogue

In [32]:
columns = ["help_id", "field", "ra", "dec", "hp_idx"]

bands = [column[5:] for column in master_catalogue.colnames if 'f_ap' in column]
for band in bands:
    columns += ["f_ap_{}".format(band), "ferr_ap_{}".format(band),
                "m_ap_{}".format(band), "merr_ap_{}".format(band),
                "f_{}".format(band), "ferr_{}".format(band),
                "m_{}".format(band), "merr_{}".format(band),
                "flag_{}".format(band)]    
    
columns += ["flag_cleaned", "flag_gaia", 
            "flag_optnir_obs", "flag_optnir_det", "ebv"] #"stellarity", "flag_merged"

In [33]:
# We check for columns in the master catalogue that we will not save to disk.
print("Missing columns: {}".format(set(master_catalogue.colnames) - set(columns)))

Missing columns: set()


In [34]:
master_catalogue[columns].write("{}/master_catalogue_spire-nep{}.fits".format(OUT_DIR, SUFFIX))